In [85]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing

train=pd.read_csv('train.csv', index_col=0)
test=pd.read_csv('test.csv', index_col=0)

In [90]:
# Simple implementation of the (normalized) gini score in numpy
# Fully vectorized, no python loops, zips, etc.
# Significantly (>30x) faster than previous implementions

import numpy as np 

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort true values on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(1/n_samples, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return G_pred/G_true

In [87]:
# label encode the categorical variables
for i in range(train.shape[1]):
    if train.iloc[:,i].dtype=='O':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train.iloc[:,i]) + list(test.iloc[:,i-1]))
        train.iloc[:,i] = lbl.transform(train.iloc[:,i])
        test.iloc[:,i-1] = lbl.transform(test.iloc[:,i-1])

In [88]:
#let's do cross-validation
from sklearn.cross_validation import KFold
kf=KFold(train.shape[0], n_folds=10, shuffle = True)
predictors=train.columns.values.tolist()[1:]

In [91]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=10)
ginies=[]
for train_index, val_index in kf:
    neigh.fit(train[predictors].iloc[train_index,:],train.Hazard.iloc[train_index])
    predictions=neigh.predict(train[predictors].iloc[val_index,:])
    ginies.append(Gini(train.Hazard.iloc[val_index],predictions))
print(np.array(ginies).mean())

0.174880232039


In [83]:
neigh.fit(train[predictors],train['Hazard'])
predictions=neigh.predict(test[predictors])

submission = pd.DataFrame({
        "Id": test.index,
        "Hazard": predictions
    })

submission.to_csv("submission_python_knn.csv", index=False, columns=['Id','Hazard'])

KeyError: 'Id'

In [84]:
submission = pd.DataFrame({
        "Id": test.index,
        "Hazard": predictions
    })

submission.to_csv("submission_python_knn.csv", index=False, columns=['Id','Hazard'])